In [ ]:
import sys
sys.path.append('../../../')
sys.path.append('/home/sheida.rahnamai/GIT/HDN/')
import torch
import pickle
import matplotlib.pyplot as plt
import numpy as np
from tifffile import imread
import glob
import matplotlib.patches as patches
import hdbscan as hd
from hdbscan import prediction
from sklearn.cluster import HDBSCAN
from collections import Counter
from openTSNE import TSNE
from openTSNE import TSNEEmbedding
from openTSNE import affinity
from openTSNE import initialization
from sklearn.decomposition import PCA
from scipy.spatial.distance import cdist
# %reload lib.plotting 
from lib import plotting as p
from lib import dataprep as dp
import random
import umap
from tqdm import tqdm
from lib.evaluation import FeatureExtractor
# from lib.dataloader import CustomTestDataset
from lib.dataloader import CustomDataset
import tifffile as tiff
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
import os
from boilerplate import boilerplate
from importlib import reload
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as mcolors
import seaborn as sns
import pandas as pd
from lib.logging import ClassType
from sklearn.metrics import confusion_matrix, adjusted_rand_score, normalized_mutual_info_score, homogeneity_score, completeness_score, v_measure_score
from boilerplate import boilerplate as blp

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

patch_size = 64
sample_size = 300
centre_size = 4
n_channel = 32
hierarchy_level = 3
pad_size = (patch_size - centre_size) // 2
model_dir = "/group/jug/Sheida/HVAE/v07/model/"
# model_dir = "/home/sheida.rahnamai/GIT/HDN/examples/Pixel_Noise/Convallaria/testmodel/"
model = torch.load(model_dir+"HVAE_best_vae.net")

In [ ]:
# train data

data_dir = "/localscratch/data/"
Three_train_images = ['high_c1', 'high_c2', 'high_c3']

# Load source images
train_img_paths = [os.path.join(data_dir, img, f"{img}_source.tif") for img in Three_train_images]
images = {img: tiff.imread(path) for img, path in zip(Three_train_images, train_img_paths)}

# Print loaded train images paths
print("Train images loaded from paths:")
for img, path in zip(Three_train_images, train_img_paths):
   print(path)

labels = {}
# Load ground truth images
for img in Three_train_images:
   gt_path = os.path.join(data_dir, img, f"{img}_gt.tif")
   labels[img] = tiff.imread(gt_path)

# Initialize dictionaries for the split data
train_images = {}
val_images = {}
train_labels = {}
val_labels = {}

keys = ['high_c1', 'high_c2', 'high_c3']

for key in tqdm(keys, desc='Splitting data'):
   filtered_image, filtered_label = boilerplate._filter_slices(images[key], labels[key])
   train_image, val_image, train_label, val_label = boilerplate._split_slices(
      filtered_image, filtered_label
   )
   train_images[key] = train_image
   val_images[key] = val_image
   train_labels[key] = train_label
   val_labels[key] = val_label

# compute mean and std of the data
all_elements = np.concatenate([train_images[key].flatten() for key in keys])
data_mean = np.mean(all_elements)
data_std = np.std(all_elements)

# normalizing the data
for key in tqdm(keys, 'Normalizing data'):
   train_images[key] = (train_images[key] - data_mean) / data_std
   val_images[key] = (val_images[key] - data_mean) / data_std

train_set = CustomDataset(train_images, train_labels)
val_set = CustomDataset(val_images, val_labels)

One_test_image = ['high_c4']

# Load test image
test_img_path = os.path.join(data_dir, One_test_image[0], f"{One_test_image[0]}_source.tif")
test_image = tiff.imread(test_img_path)

# Print loaded test images paths
print("Test image loaded from path:")
print(test_img_path)

# Load test ground truth images
test_gt_path = os.path.join(data_dir, One_test_image[0], f"{One_test_image[0]}_gt.tif")
test_ground_truth_image = tiff.imread(test_gt_path)

bg_indices = random.sample(train_set.patches_by_label[0],sample_size)
nucleus_indices = random.sample(train_set.patches_by_label[1],sample_size)
granule_indices = random.sample(train_set.patches_by_label[2],sample_size)
mito_indices = random.sample(train_set.patches_by_label[3],sample_size)

bg_samples, bg_cls, bg_lbl = train_set[bg_indices]
nucleus_samples, nucleus_cls, nucleus_lbl = train_set[nucleus_indices]
granule_samples, granule_cls, granule_lbl = train_set[granule_indices]
mito_samples, mito_cls, mito_lbl = train_set[mito_indices]

bg_samples = bg_samples.squeeze(1)
bg_lbl = bg_lbl.squeeze(1)
nucleus_samples = nucleus_samples.squeeze(1)
nucleus_lbl = nucleus_lbl.squeeze(1)
granule_samples = granule_samples.squeeze(1)
granule_lbl = granule_lbl.squeeze(1)
mito_samples = mito_samples.squeeze(1)
mito_lbl = mito_lbl.squeeze(1)

#test data

test_images = (test_image-data_mean)/data_std

test_slice = test_images[626]
test_slice_lbl = test_ground_truth_image[626]

In [ ]:
prediction = tiff.imread('../../../feature_maps.tif')
clusters = tiff.imread('clusters.tif')
x, y = prediction.shape
pred = np.copy(prediction)
v, w = clusters.shape
clust = np.copy(clusters)
test_slice_lbl[30:30+x,30:30+y].shape
pred[test_slice_lbl[30:30+x,30:30+y] == -1] = -1
clust[test_slice_lbl[30:30+x,30:30+y] == -1] = -1
clust[clust == 1] = 4
clust[clust == 2] = 1
clust[clust == 4] = 2
cropped_gt = test_slice_lbl[30:30+x,30:30+y]

In [ ]:
p.plot_w_b(
        (test_slice, "Input cropped", (pad_size, pad_size, x, y)),\
        (test_slice_lbl, "GT label cropped", (pad_size, pad_size, x, y)),\
        (clust, "clusters", False),\
        (pred, "Prediction of cell", False),\
        plot_types=['imshow', 'imshow_l', 'imshow_l', 'imshow_l'],\
        box_size=10)

In [ ]:
from sklearn.metrics import adjusted_rand_score
gt_flat = cropped_gt.flatten()
pred_flat = pred.flatten()
clust_flat = clust.flatten()
ari_score1 = adjusted_rand_score(gt_flat, pred_flat)
ari_score2 = adjusted_rand_score(gt_flat, clust_flat)
print(f'pred based on train {ari_score1}')
print(f'pred based on clustering {ari_score2}')

In [ ]:
def dice_coefficient_per_class(y_true, y_pred, class_label):
    intersection = np.sum((y_true == class_label) & (y_pred == class_label))
    return 2 * intersection / (np.sum(y_true == class_label) + np.sum(y_pred == class_label))

def dice_coefficient_multilabel(y_true, y_pred, num_classes, ignore_label=-1):
    dice_scores = []
    for class_label in range(num_classes):
        if class_label == ignore_label:
            continue
        dice = dice_coefficient_per_class(y_true, y_pred, class_label)
        dice_scores.append(dice)
    return np.array(dice_scores)

num_classes = 4  # Number of labels/classes (excluding the -1 label)

dsc_per_class1 = dice_coefficient_multilabel(cropped_gt, pred, num_classes)
dsc_per_class2 = dice_coefficient_multilabel(cropped_gt, clust, num_classes)
print(f"Train Dice Similarity Coefficient per class: {dsc_per_class1}")
print(f"Cluster Dice Similarity Coefficient per class: {dsc_per_class2}")

print(f"Train Mean Dice Similarity Coefficient: {np.mean(dsc_per_class1):.4f}")
print(f"Cluster Mean Dice Similarity Coefficient: {np.mean(dsc_per_class2):.4f}")

In [ ]:
from sklearn.metrics import f1_score

# Filter out the background label (-1)
mask = gt_flat != -1
y_true_filtered = gt_flat[mask]
y_pred_filtered1 = pred_flat[mask]
y_pred_filtered2 = clust_flat[mask]

# Calculate the Dice coefficient as the F1 score for each class
dsc1 = f1_score(y_true_filtered, y_pred_filtered1, average=None)
dsc2 = f1_score(y_true_filtered, y_pred_filtered2, average=None)

print(f"Train Dice Similarity Coefficient per class: {dsc1}")
print(f"Cluster Dice Similarity Coefficient per class: {dsc2}")

print(f"Train Mean Dice Similarity Coefficient: {np.mean(dsc1):.4f}")
print(f"Cluster Mean Dice Similarity Coefficient: {np.mean(dsc2):.4f}")